# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys_geoapify import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.88,87,10,9.99,PN,1706813738
1,1,isangel,-19.5500,169.2667,24.06,88,100,3.60,VU,1706813738
2,2,tazovsky,67.4667,78.7000,-12.25,94,100,7.81,RU,1706813739
3,3,bodden town,19.2833,-81.2500,25.55,62,20,5.66,KY,1706813739
4,4,ushuaia,-54.8000,-68.3000,18.81,34,40,7.72,AR,1706813604


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.5,
    color = 'City'
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
]
ideal_weather_df.head()
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,hawaiian paradise park,19.5933,-154.9731,26.04,70,0,2.57,US,1706813761
254,254,martaban,16.5314,97.6111,23.56,75,0,1.18,MM,1706813815
282,282,myeik,12.4333,98.6000,24.76,82,0,1.74,MM,1706813824
283,283,yanchep,-31.5500,115.6833,26.64,59,0,2.86,AU,1706813824
439,439,alice springs,-23.7000,133.8833,22.34,21,0,3.60,AU,1706813725
452,452,diboll,31.1871,-94.7810,22.15,49,0,2.06,US,1706813876
516,516,chokwe,-24.5333,32.9833,24.20,77,0,4.32,MZ,1706813865


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
81,hawaiian paradise park,US,19.5933,-154.9731,70,
254,martaban,MM,16.5314,97.6111,75,
282,myeik,MM,12.4333,98.6000,82,
283,yanchep,AU,-31.5500,115.6833,59,
439,alice springs,AU,-23.7000,133.8833,21,
452,diboll,US,31.1871,-94.7810,49,
516,chokwe,MZ,-24.5333,32.9833,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
#import urllib.parse

# Set parameters to search for a hotel
radius = 10000 

params = {
    "radius": radius,
    "apiKey": geoapify_key,
    "categories": 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"

    #url = f"{base_url}?{urllib.parse.urlencode(params)}"
    #print(f"Request URL: {url}")
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
martaban - nearest hotel: မော်​လ​မြိုင် ဟိုတယ်
myeik - nearest hotel: Green Eyes Hotel
yanchep - nearest hotel: Yanchep Inn
alice springs - nearest hotel: Aurora Alice Springs
diboll - nearest hotel: No hotel found
chokwe - nearest hotel: Limpopo Hotel


,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
81,hawaiian paradise park,US,19.5933,-154.9731,70,No hotel found,No hotel found
254,martaban,MM,16.5314,97.6111,75,No hotel found,မော်​လ​မြိုင် ဟိုတယ်
282,myeik,MM,12.4333,98.6000,82,No hotel found,Green Eyes Hotel
283,yanchep,AU,-31.5500,115.6833,59,No hotel found,Yanchep Inn
439,alice springs,AU,-23.7000,133.8833,21,No hotel found,Aurora Alice Springs
452,diboll,US,31.1871,-94.7810,49,No hotel found,No hotel found
516,chokwe,MZ,-24.5333,32.9833,77,No hotel found,Limpopo Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 1.0,
    color = 'City',
    hover_cols=['City', 'Hotel Name', 'Country']
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)